In [45]:
# 1. 라이브러리 불러오기
import cv2 # OpenCV는 이미지 및 동영상 처리에 사용되는 라이브러리입니다
from ultralytics import YOLO # ultralytics는 YOLO 객체 탐지 모델을 쉽게 사용할 수 있게 해주는 라이브러리입니다

In [46]:
# 2. 동영상 파일 경로 지정
# 분석하고자 하는 동영상 파일의 경로를 지정합니다
video_path = 'incredibles.mp4' # 'incredibles.mp4'는 분석할 동영상 파일 이름입니다

In [47]:
# VideoCapture() 함수는 지정한 동영상 파일을 불러오는 함수입니다
# 매개변수:
#  - video_path: 불러올 동영상 파일의 경로 또는 이름입니다
cap = cv2.VideoCapture(video_path)

In [48]:
# 3. YOLO 모델 불러오기
# YOLO는 사람이나 사물 등의 객체를 실시간으로 탐지하는 모델입니다
# 아래는 사전에 학습된 YOLO 모델 파일을 불러오는 부분입니다

# YOLO() 함수는 YOLO 모델을 불러와 객체 탐지를 할 수 있도록 준비합니다
# 매개변수:
#  - 'incredible_yolo_model.pt': 사전에 학습된 YOLO 모델 파일 경로입니다
model = YOLO('incredible_yolo_model.pt')

In [49]:
# 4. 원본 동영상의 속성(크기, 초당 프레임 수) 정보 얻기
# 동영상 파일의 너비, 높이, 초당 프레임 수(FPS)를 가져와서 새로운 동영상을 저장할 때 사용합니다
# 나중에 결과를 저장할 때 똑같은 크기와 속도로 만들어야 하기 때문입니다

# cap.get() 함수는 동영상에서 원하는 정보를 가져올 수 있습니다
# CAP_PROP_FRAME_WIDTH: 프레임의 가로 너비를 가져옵니다
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 프레임의 너비입니다

width

1336

In [50]:
# CAP_PROP_FRAME_HEIGHT: 프레임의 세로 높이를 가져옵니다
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 프레임의 높이입니다

height

568

In [51]:
# CAP_PROP_FPS: 동영상의 초당 프레임 수(FPS)를 가져옵니다
fps = cap.get(cv2.CAP_PROP_FPS) # 초당 몇 장의 이지미가 재생되는지를 의미합니다

fps

25.645900792308844

In [52]:
# 5. 결과를 저장할 VideoWriter 설정
# 객체 탐지 후 결과를 새로운 동영상 파일로 저장히기 위한 준비입니다

# fourcc는 비디오 코덱(four character code)을 지정하는 함수입니다
# 코덱은 압축 방식이며, 어떤 형식으로 동영상을 저장할지를 결정합니다
# 'mp4v'는 MP4 포맷의 코덱이며, 대부분의 플레이어에서 잘 작동합니다
# 예시: 'XVID', 'MJPG', 'DIVX', 'mp4v' 등 다양한 코덱이 있습니다
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

In [53]:
# VideoWrtier() 함수는 새로운 동영상 파일을 생성할 때 사용합니다
# 매개변수:
# - 파일 이름: 'output_detected.mp4'로 저장합니다
# - fourcc: 사용할 코덱입니다
# - fps: 초당 프레임 수입니다
# - (width, height): 프레임의 크기입니다

# 기존 동영상 incredibles.mp4와 똑같은 (width, height)와 fps입니다
out = cv2.VideoWriter('output_detected.mp4', fourcc, fps, (width, height))

In [54]:
# 6. 동영상을 한 프레임씩 읽어가며 객체 탐지 및 저장
# cap.inOpened()는 동영상이 정상적으로 열렸는지 확인합니다
# 동영상이 정상적으로 열렸으면 True가 리턴되고 반복문이 실행됩니다
while cap.isOpened():
    # cap.read()는 프레임을 하니씩 읽습니다
    # 반환값:
    # - ret: 프레임을 제대로 읽었는지 여부(True 또는 False)
    #        더 이상 읽을 프레임이 없으면(동영상 마기작까지 다 읽음 False를 리턴합니다)
    # - frame: 읽어온 실제 이미지(프레임)입니다
    ret, frame = cap.read()

    # ret가 False이면
    # not ret는 반대로 True를 리턴합니다
    # not ret가 True 일때 즉 더 이상 읽을 프레임이 없으면 반복을 종료합니다
    if not ret:
        break

    # model() 함수는 입력된 이미지를 분석하여 객체를 탐지합니다
    # 매개변수:
    # - frame: 분석할 이미지입니다
    # - conf=0.5: 최소 신뢰도 50% 이상인 객체만 탐지합니다
    results = model(frame, conf=0.5)

    # results[0].plot() 함수는 탐지된 객체(박스와 이름 등)를 이미지 위에 시각화하여 보여줍니다
    # 결과는 새로운 이미지(annotated_frame)로 반환됩니다
    annotated_frame = results[0].plot()

    # out.write() 함수는 위에서 생성한 annotated_frame을 동영상 파일에 저장합니다
    out.write(annotated_frame)

# 7. 모든 작업이 끝난 후 자원 해제
# 동영상 파일과 저장 장치를 메모리에서 해제합니다
cap.release() # 동영상 파일을 닫습니다
out.release() # 저장 중인 동영상 파일을 마칩니다

# 완료 메시지를 출력합니다
print("탐지 결과가 'output_detected.mp4.'로 저장되었습니다")


0: 288x640 1 mr_incredible, 1 elastic_girl, 33.4ms
Speed: 3.6ms preprocess, 33.4ms inference, 8.4ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 mr_incredible, 1 elastic_girl, 12.6ms
Speed: 2.0ms preprocess, 12.6ms inference, 1.3ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 mr_incredible, 1 elastic_girl, 15.0ms
Speed: 1.5ms preprocess, 15.0ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 mr_incredible, 2 dashs, 9.4ms
Speed: 1.5ms preprocess, 9.4ms inference, 1.3ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 mr_incredible, 1 dash, 2 violets, 16.2ms
Speed: 1.6ms preprocess, 16.2ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 mr_incredible, 1 dash, 2 violets, 10.8ms
Speed: 1.4ms preprocess, 10.8ms inference, 1.3ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 dash, 7.0ms
Speed: 1.2ms preprocess, 7.0ms inference, 1.2ms postprocess per image at shape (1, 